In [1]:
%load_ext sql
%sql postgresql://appdev@data:5432/appdev

'Connected: appdev@appdev'

In [78]:
%sql SELECT * from information_schema.tables WHERE table_schema = 'f1db';

13 rows affected.


table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
appdev,f1db,circuits,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructorresults,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructors,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructorstandings,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,drivers,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,driverstandings,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,laptimes,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,pitstops,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,qualifying,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,races,BASE TABLE,None,None,None,None,None,YES,NO,None


# Scaling PostgreSQL
## Database Assignment 7 (Not Mandatory)

### 1. Using a single join and a grouping, write a query that allows you to find the forename, surname, driverid and the total number of wins for each driver in the f1db schema.

In [121]:
%%sql SELECT 
    drivers.driverid as driverId,
    drivers.forename as forename,
    drivers.surname as surname,
    count(results.position) as wins
FROM drivers
JOIN results USING (driverid)
WHERE results.position = 1 
GROUP BY (drivers.driverid) 
ORDER BY wins DESC
LIMIT 5;

5 rows affected.


driverid,forename,surname,wins
30,Michael,Schumacher,91
1,Lewis,Hamilton,60
117,Alain,Prost,51
20,Sebastian,Vettel,46
102,Ayrton,Senna,41


### 2. Write a query that joins the tables results, constructors and drivers to show the amount of times a driver has driven a car from a constructor. In other words how many times one driver (for instance Schumacher) has driven a constructor (for instance Mercedes). Your table should have three fields: drivers.driverref, constructors.name and count(*) (showing the number of times the driver has driven the constructor).
Hint: You can group on more than one value

In [122]:
%%sql SELECT 
    drivers.driverref as driver,
    constructors.name as constructor,
    count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
GROUP BY (constructors.name, drivers.driverref) 
ORDER BY time DESC
LIMIT 5;

5 rows affected.


driver,constructor,time
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136


### 3. Now we have found the most driven vehicles even for those who didn't finish the race. Extend the query from 2 by removing all tuples from the result where the status is not "Finished".

In [123]:
%%sql SELECT 
    drivers.driverref as driver,
    constructors.name as constructor,
    count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY (constructors.name, drivers.driverref) 
ORDER BY time DESC
LIMIT 5;

5 rows affected.


driver,constructor,time
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97


### 4. Create a third (and new) query that finds the amount of milliseconds spent in pitstop (see the pitstops relation) by each unique combinations of driverid and raceid.
Hint: use the sum aggregate function to find the total amount of milliseconds
Hint: i recommend making this into a view

In [125]:
%%sql CREATE VIEW pitstop_times AS SELECT 
    drivers.driverid as driver_id,
    races.raceid as race_id,
    count(pitstops.driverid) as times,
    sum(pitstops.milliseconds) as milliseconds
FROM drivers
JOIN pitstops USING (driverid)
JOIN races USING (raceid)
GROUP BY (drivers.driverid, races.raceid);

Done.


[]

In [128]:
%sql SELECT * FROM pitstop_times ORDER BY milliseconds DESC LIMIT 5 ;

5 rows affected.


driver_id,race_id,times,milliseconds
817,967,5,3572090
18,967,5,3569001
830,967,5,3563738
13,967,4,3554774
826,967,4,3546767


### 5. Time to put everything together. Use the query for 4 as a join subquery in query 3 (see slides on "a query within a query") to find the total amount of pitstop time, for each result in query 3

In [144]:
%%sql SELECT 
    drivers.driverref as driver,
    constructors.name as constructor,
    count(*) as time,
    pitstop_times.times as pitstop_times,
    pitstop_times.millisecond as pitstop_milliseconds
FROM drivers
INNER JOIN (SELECT * FROM pitstop_times) as pitstop_times
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1 and 
GROUP BY (constructors.name, drivers.driverref) 
ORDER BY time DESC
LIMIT 5;

(psycopg2.ProgrammingError) syntax error at or near "WHERE"
LINE 11: WHERE results.statusid = 1
         ^
 [SQL: 'SELECT \n    drivers.driverref as driver,\n    constructors.name as constructor,\n    count(*) as time,\n    pitstop_times.times as pitstop_times,\n    pitstop_times.millisecond as pitstop_milliseconds\nFROM drivers\nINNER JOIN (SELECT * FROM pitstop_times) as pitstop_times\nJOIN results USING (driverid)\nJOIN constructors USING (constructorid)\nWHERE results.statusid = 1\nGROUP BY (constructors.name, drivers.driverref) \nORDER BY time DESC\nLIMIT 5;']


### 6. As a last thing we want to find the driver with the least pistop time. However we want to control for the amount of races (see explanation below), so we need to include one last column: the average pitstop time per race. Your table should be sorted based on the ascending values of that average pitstop time.
    A. Explanation: If we just count the pitstop time, we only get the total time he spend in a workshop, no matter how many races he participated in. If a driver participated in 100 races, he would have more pitstop time than a driver participating in one race!
    B. Hint: Use the sum function to accumulate the total pistop time and divide that with the amount of races the driver has participated in